# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### VectorDB Instance

In [5]:
chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path="chromadb")
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=CHROMA_OPENAI_API_KEY)

### Collection

In [6]:
collection = chroma_client.get_or_create_collection(
   name="udaplay",
   embedding_function=embedding_fn
)

### Add documents

In [7]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

if not os.path.exists(data_dir):
    print(f"Error: {data_dir} does not exist in {os.getcwd()}")
    exit(1)

print(f"Indexing games from {data_dir}...")
count = 0
for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )
    print(f"Indexed: {game['Name']}")
    count += 1
    
print(f"Finished indexing {count} games.")

Indexing games from games...
Indexed: Gran Turismo
Indexed: Grand Theft Auto: San Andreas
Indexed: Gran Turismo 5
Indexed: Marvel's Spider-Man
Indexed: Marvel's Spider-Man 2
Indexed: Pokémon Gold and Silver
Indexed: Pokémon Ruby and Sapphire
Indexed: Super Mario World
Indexed: Super Mario 64
Indexed: Super Smash Bros. Melee
Indexed: Wii Sports
Indexed: Mario Kart 8 Deluxe
Indexed: Kinect Adventures!
Indexed: Minecraft
Indexed: Halo Infinite
Finished indexing 15 games.
